In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("/Users/bytedance/MLTrader-Research/")
import sys
sys.path.insert(0, ".")
import pandas as pd
from research.orderbook_strategies.utils.factor_analysis import update_signal_results_db
from research.orderbook_strategies.utils.helper import load
import numpy as np
from scipy.stats import spearmanr
import research.orderbook_strategies.utils.helper as helper
import research.orderbook_strategies.utils.factor_analysis as factor_analysis

In [3]:
# factor_analysis.update_signal_results_db('signal_result_with_atr')

In [4]:
period_list = [32, 64, 128, 256, 512, 1024, 2048]
dire_signal_list = ["kdj.j", "buy_power", "adx.direc2.1024", "rmi", "volume.advantage", 
                    "kdj.k", "ma.dif.10", "dbook", "range.pos", "nr", 
                    "dbook.spread.diff", "dbook.spread"]
dire_signal_list_period = [signal+".{}".format(period) for period in period_list for signal in dire_signal_list]
range_signal_list = ["std", "trend.index", "atr", "adx.1024"]

dire_signal_with_variance = np.array([])
dire_signal_with_variance_period = []
for dire_signal in dire_signal_list:
    for range_signal in range_signal_list:
        signal_name = dire_signal+"."+range_signal +".2048"
        dire_signal_with_variance = np.append(dire_signal_with_variance, signal_name)
dire_signal_with_variance = list(dire_signal_with_variance)
dire_signal_with_variance_period = [signal+".{}".format(period) for period in period_list for signal in dire_signal_with_variance]
trades_signal = ['trade.buy_power.more_position.12', 'trade.doublebuy.value.ratio', 'trade.buy_power', 'trade.buy_power_count', 'trade.buy_power']
trades_signal_period = [signal+".{}".format(period) for period in np.power(2, range(10, 15)) for signal in trades_signal]

print("#dire = {}, #dire_with_var = {}, #trades = {}".format(len(dire_signal_list_period), len(dire_signal_with_variance_period), len(trades_signal_period)))

#dire = 84, #dire_with_var = 336, #trades = 25


In [5]:
product_list = ["OPUSDT","BTCUSDT", "ETHUSDT", "APTUSDT", "1000PEPEUSDT"]
all_results = pd.read_csv("/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal.csv")

all_signals = all_results['signal_name'].unique()
for signal_name in dire_signal_list_period:
    if signal_name not in all_signals:
        print("{} not exist".format(signal_name))
        
for signal_name in dire_signal_with_variance_period:
    if signal_name not in all_signals:
        print("{} not exist".format(signal_name))
        
for signal_name in trades_signal_period:
    if signal_name not in all_signals:
        print("{} not exist".format(signal_name))

rmi.adx.1024.2048.2048 not exist


In [6]:
grouper_names = ["grouper_0", "grouper_1", "grouper_2", "grouper_3"]
all_results["grouper_0"] = all_results["signal_name"].astype(str) + "_" + all_results["reverse"].astype(str)
all_results["grouper_1"] = all_results["grouper_0"] + "_" + all_results["product"].astype(str)
all_results["grouper_2"] = all_results["grouper_1"] + "_" + all_results["open"].astype(str)
all_results["grouper_3"] = all_results["grouper_2"] + "_" + all_results["close"].astype(str)

## 先filter 交易次数过少的因子
stats = all_results.pivot_table(index="grouper_0", columns="is_train", aggfunc="median", values='avg.num')
prev = len(stats.index)
valid_str = list(stats[stats[True] > 1].index)
now_have = len(valid_str)
all_results = all_results.query("grouper_0 in @valid_str")
print("keep {}/{}".format(now_have, prev))

keep 635/950


### Learderboard for Trades Signal Top20 (全Symbol中位数)

In [9]:
sub_res['avg.num'].groupby(by=[sub_res['grouper_0'], sub_res["is_train"]], dropna=False).median()

grouper_0                               is_train
trade.doublebuy.value.ratio.1024_-1     False       18.818898
                                        True        27.330472
trade.doublebuy.value.ratio.1024_-1.0   False       26.213178
                                        True        86.107362
trade.doublebuy.value.ratio.1024_1      False       18.818898
                                        True        27.330472
trade.doublebuy.value.ratio.1024_1.0    False       26.213178
                                        True        86.107362
trade.doublebuy.value.ratio.16384_-1    False        3.602362
                                        True         4.045064
trade.doublebuy.value.ratio.16384_-1.0  False        2.638462
                                        True        13.200000
trade.doublebuy.value.ratio.16384_1     False        3.602362
                                        True         4.045064
trade.doublebuy.value.ratio.16384_1.0   False        2.638462
                     

In [10]:
# group_0
sub_res = all_results[np.isin(all_results["signal_name"], trades_signal_period)]
cols_of_interest = ["sharpe.ret", "avg.num", "avg.ret"]
group_0_res = sub_res.groupby(by=["grouper_0", "is_train"], dropna=False)[cols_of_interest].median()
group_0_res_train = group_0_res.loc[(slice(None), True), :].droplevel(1)
group_0_res_test = group_0_res.loc[(slice(None), False), :].droplevel(1)

good_groups_in_train = list(group_0_res_train[group_0_res_train['sharpe.ret'] > 0].index)
print("#good_in_train: {}/{}".format(len(good_groups_in_train), group_0_res_train.shape[0]))

test_perf_of_good = group_0_res_test[np.isin(group_0_res_test.index, good_groups_in_train)]
test_perf_of_good["sharpe.ret.train"] = test_perf_of_good.index.map(group_0_res_train['sharpe.ret'])

# performance without transaction cost
print("Learderboard for Trades Signal Top20 (全Symbol中位数)")
display(test_perf_of_good.sort_values(by="sharpe.ret.train", ascending=False).head(20))

#good_in_train: 8/20
Learderboard for Trades Signal Top20 (全Symbol中位数)


,sharpe.ret,sharpe.ret.train
grouper_0,,
trade.doublebuy.value.ratio.2048_1,0.342844,1.159348
trade.doublebuy.value.ratio.8192_1,-0.502210,1.026112
trade.doublebuy.value.ratio.16384_1.0,-1.402061,0.925750
trade.doublebuy.value.ratio.4096_1,0.720014,0.823371
trade.doublebuy.value.ratio.16384_1,-0.107041,0.795890
trade.doublebuy.value.ratio.8192_1.0,-1.400780,0.564425
trade.doublebuy.value.ratio.1024_1,-1.380805,0.084600
trade.doublebuy.value.ratio.4096_1.0,-1.292176,0.018846


### Learderboard for Direc Signal Top20 (全Symbol中位数)

In [8]:
# group_0
sub_res = all_results[np.isin(all_results["signal_name"], dire_signal_list_period)]
group_0_res = sub_res.groupby(by=["grouper_0", "is_train"]).median()
group_0_res_train = group_0_res.loc[(slice(None), True), :].droplevel(1)
group_0_res_test = group_0_res.loc[(slice(None), False), :].droplevel(1)

cols_of_interest = ["sharpe.ret", "avg.num", "avg.ret"]
good_groups_in_train = list(group_0_res_train[group_0_res_train['sharpe.ret'] > 0].index)
print("#good_in_train: {}/{}".format(len(good_groups_in_train), group_0_res_train.shape[0]))

test_perf_of_good = group_0_res_test[np.isin(group_0_res_test.index, good_groups_in_train)][cols_of_interest]
test_perf_of_good["sharpe.ret.train"] = test_perf_of_good.index.map(group_0_res_train['sharpe.ret'])

# performance without transaction cost
print("Learderboard for Direc Signal Top20 (全Symbol中位数)")
test_perf_of_good.sort_values(by="sharpe.ret.train", ascending=False).head(20)

#good_in_train: 43/114


,sharpe.ret,avg.num,avg.ret,sharpe.ret.train
grouper_0,,,,
range.pos.64_-1,0.144861,7.073643,0.000012,1.452959
volume.advantage.32_-1,0.161001,19.315891,0.000011,1.295063
kdj.j.128_-1,0.263086,6.264210,0.000030,1.197517
kdj.k.128_-1,-0.124558,9.282946,-0.000006,1.017383
volume.advantage.64_-1,0.290576,11.732558,0.000022,0.825879
volume.advantage.256_-1,0.420705,4.183764,0.000073,0.787381
rmi.32_-1,-0.213175,11.570847,-0.000017,0.737305
rmi.64_-1,0.177408,6.888814,0.000022,0.684367
kdj.k.256_-1,0.305607,5.847436,0.000028,0.585335


### Learderboard for Direc Signal Top20 (分Symbol中位数)

In [9]:
# group_0
sub_res = all_results[np.isin(all_results["signal_name"], dire_signal_list_period)]
group_0_res = sub_res.groupby(by=["grouper_1", "is_train"]).median()
group_0_res_train = group_0_res.loc[(slice(None), True), :].droplevel(1)
group_0_res_test = group_0_res.loc[(slice(None), False), :].droplevel(1)

cols_of_interest = ["sharpe.ret", "avg.num", "avg.ret"]
good_groups_in_train = list(group_0_res_train[group_0_res_train['sharpe.ret'] > 0].index)
print("#good_in_train: {}/{}".format(len(good_groups_in_train), group_0_res_train.shape[0]))

test_perf_of_good = group_0_res_test[np.isin(group_0_res_test.index, good_groups_in_train)][cols_of_interest]
test_perf_of_good["sharpe.ret.train"] = test_perf_of_good.index.map(group_0_res_train['sharpe.ret'])
test_perf_of_good['product'] = pd.Series(test_perf_of_good.index).map(lambda x: x.split("_")[-1]).values
# performance without transaction cost
for product in product_list:
    display(test_perf_of_good[test_perf_of_good['product'] == product].sort_values(by="sharpe.ret.train", ascending=False).head(10))

#good_in_train: 212/570


,sharpe.ret,avg.num,avg.ret,sharpe.ret.train,product
grouper_1,,,,,
kdj.k.512_-1_OPUSDT,0.173561,5.521318,0.000030,1.537544,OPUSDT
range.pos.64_-1_OPUSDT,-0.836934,10.422481,-0.000062,1.255256,OPUSDT
kdj.k.1024_-1_OPUSDT,0.624984,2.798450,0.000160,1.237924,OPUSDT
range.pos.256_-1_OPUSDT,0.040463,3.705426,0.000007,1.226860,OPUSDT
kdj.j.512_-1_OPUSDT,0.472893,3.321705,0.000118,1.184801,OPUSDT
kdj.j.128_-1_OPUSDT,-0.506398,9.608527,-0.000038,1.129159,OPUSDT
range.pos.512_-1_OPUSDT,0.473693,2.023256,0.000121,1.066582,OPUSDT
range.pos.128_-1_OPUSDT,-1.503431,6.127907,-0.000467,0.947739,OPUSDT
nr.2048_-1_OPUSDT,1.233376,2.488372,0.000998,0.756447,OPUSDT


,sharpe.ret,avg.num,avg.ret,sharpe.ret.train,product
grouper_1,,,,,
rmi.32_-1_BTCUSDT,0.944657,0.615385,0.000132,2.462280,BTCUSDT
dbook.spread.diff.32_1_BTCUSDT,1.901988,51.292308,0.000020,2.329285,BTCUSDT
range.pos.64_-1_BTCUSDT,0.687009,0.138462,0.000179,1.757135,BTCUSDT
dbook.32_1_BTCUSDT,2.680085,4.926923,0.000128,1.735396,BTCUSDT
rmi.64_-1_BTCUSDT,0.510273,1.053846,0.000176,1.733843,BTCUSDT
volume.advantage.32_-1_BTCUSDT,3.098785,3.373077,0.000130,1.485010,BTCUSDT
kdj.j.64_-1_BTCUSDT,1.738645,0.138462,0.000169,1.373965,BTCUSDT
kdj.k.128_-1_BTCUSDT,0.135146,0.069231,0.000037,1.362593,BTCUSDT
dbook.spread.128_-1_BTCUSDT,-0.527665,0.034615,-0.000110,1.358282,BTCUSDT


,sharpe.ret,avg.num,avg.ret,sharpe.ret.train,product
grouper_1,,,,,
kdj.j.128_-1_ETHUSDT,1.540277,0.192308,0.000129,2.466401,ETHUSDT
ma.dif.10.32_-1_ETHUSDT,1.883586,5.153846,0.000080,2.432456,ETHUSDT
range.pos.64_-1_ETHUSDT,0.916314,0.492308,0.000133,2.319376,ETHUSDT
kdj.k.256_-1_ETHUSDT,1.371437,0.430769,0.000116,2.142311,ETHUSDT
nr.128_-1_ETHUSDT,0.653119,0.419231,0.000095,2.139976,ETHUSDT
dbook.32_1_ETHUSDT,3.127482,8.057692,0.000091,2.109414,ETHUSDT
volume.advantage.32_-1_ETHUSDT,0.561334,2.496154,0.000028,2.080575,ETHUSDT
kdj.k.128_-1_ETHUSDT,3.276297,0.265385,0.000450,2.070209,ETHUSDT
range.pos.32_-1_ETHUSDT,3.651169,0.753846,0.000348,1.978111,ETHUSDT


,sharpe.ret,avg.num,avg.ret,sharpe.ret.train,product
grouper_1,,,,,
rmi.256_1_APTUSDT,-1.082750,2.545802,-0.000245,1.539504,APTUSDT
adx.direc2.1024.512_1_APTUSDT,0.000000,3.702290,0.000000,1.335014,APTUSDT
adx.direc2.1024.256_1_APTUSDT,-0.031388,4.396947,-0.000007,1.301371,APTUSDT
kdj.j.512_1_APTUSDT,-1.277332,2.912214,-0.000231,1.234922,APTUSDT
volume.advantage.1024_1_APTUSDT,-1.008771,1.927481,-0.000670,1.192847,APTUSDT
volume.advantage.32_-1_APTUSDT,0.099791,21.045802,0.000007,1.128692,APTUSDT
kdj.k.1024_1_APTUSDT,0.614063,2.656489,0.000315,1.054474,APTUSDT
rmi.128_1_APTUSDT,-0.164621,4.717557,-0.000077,1.014603,APTUSDT
rmi.512_1_APTUSDT,-0.398768,1.438931,-0.000164,0.904576,APTUSDT


,sharpe.ret,avg.num,avg.ret,sharpe.ret.train,product
grouper_1,,,,,
range.pos.512_1_1000PEPEUSDT,-0.858689,2.408333,-0.000516,2.484670,1000PEPEUSDT
kdj.k.512_1_1000PEPEUSDT,0.028898,5.866667,0.000005,2.454581,1000PEPEUSDT
dbook.spread.32_1_1000PEPEUSDT,1.228112,4.366667,0.000253,2.358948,1000PEPEUSDT
nr.64_-1_1000PEPEUSDT,-0.404082,23.700000,-0.000049,2.332312,1000PEPEUSDT
dbook.32_1_1000PEPEUSDT,1.787961,57.050000,0.000061,2.277777,1000PEPEUSDT
nr.128_-1_1000PEPEUSDT,0.180498,16.008333,0.000023,2.205314,1000PEPEUSDT
range.pos.256_1_1000PEPEUSDT,0.828051,3.691667,0.000177,2.163583,1000PEPEUSDT
kdj.j.256_1_1000PEPEUSDT,-2.208344,6.166667,-0.000258,2.059400,1000PEPEUSDT
volume.advantage.256_-1_1000PEPEUSDT,-0.021765,7.475000,-0.000007,2.047828,1000PEPEUSDT


### Learderboard for Interaction Signal Top20 (全Symbol中位数)

# (Direc Signal)构造Portfolio后的Test Return Learderboard

In [18]:
results = factor_analysis.get_signal_performance_result(dire_signal_list_period, "signal_result_with_atr", product_list, 1e-4, 1)

In [19]:
trend_res = results['trend.signal.stat']
reve_res = results["reverse.signal.stat"]
signal_trend_res = trend_res[['signal', 'trainSharpe']].set_index('signal')['trainSharpe']
signal_rever_res = reve_res[['signal', 'trainSharpe']].set_index('signal')['trainSharpe']

trend_signals = signal_trend_res[(signal_trend_res - signal_rever_res) > 0].index
reverse_signals = signal_trend_res[(signal_trend_res - signal_rever_res) < 0].index
nouse_signals = signal_trend_res[(signal_trend_res - signal_rever_res) == 0].index

print("#trend={}, #reverse={}, #nouse={}".format(len(trend_signals), len(reverse_signals), len(nouse_signals)))

merge_res = pd.concat([trend_res.loc[np.isin(trend_res['signal'], trend_signals)], 
                       reve_res.loc[np.isin(reve_res['signal'], reverse_signals)]], 
                      axis=0)
merge_res = merge_res.sort_values(by='trainSharpe', ascending=False)
display(merge_res.sort_values(by='trainSharpe', ascending=False))

#trend=38, #reverse=36, #nouse=10


,signal,reverse,num_good_product,good_product,trainSharpe,testSharpe,trainAvgDailyTradeNum,testAvgDailyTradeNum,sum_good_strat,avgTrainRet,avgTestRet
5,kdj.k.32,-1,2,"[BTCUSDT, ETHUSDT]",4.229345,2.789503,50.783587,38.246154,NaN,0.003965,0.001580
24,kdj.j.128,-1,4,"[OPUSDT, BTCUSDT, ETHUSDT, 1000PEPEUSDT]",3.666122,1.707982,58.797775,39.661869,NaN,0.003684,0.000861
17,kdj.k.64,-1,3,"[BTCUSDT, ETHUSDT, 1000PEPEUSDT]",3.631148,1.868213,76.679746,49.007543,NaN,0.004547,-0.000224
20,range.pos.64,-1,5,"[OPUSDT, BTCUSDT, ETHUSDT, APTUSDT, 1000PEPEUSDT]",3.563911,2.062242,135.552122,95.312854,NaN,0.004581,0.000574
29,kdj.k.128,-1,5,"[OPUSDT, BTCUSDT, ETHUSDT, APTUSDT, 1000PEPEUSDT]",3.556452,1.641319,145.497059,103.410684,NaN,0.004715,0.000332
...,...,...,...,...,...,...,...,...,...,...,...
69,nr.1024,1,2,"[APTUSDT, 1000PEPEUSDT]",1.067901,-2.159308,5.665777,7.341869,25.0,0.003248,-0.006460
76,volume.advantage.2048,-1,4,"[OPUSDT, BTCUSDT, ETHUSDT, 1000PEPEUSDT]",1.037994,0.896804,13.546973,11.816711,NaN,0.003123,0.000970
80,range.pos.2048,1,5,"[OPUSDT, BTCUSDT, ETHUSDT, APTUSDT, 1000PEPEUSDT]",0.979423,-0.132015,17.897810,15.878798,53.0,0.002920,0.000139
78,ma.dif.10.2048,1,4,"[OPUSDT, BTCUSDT, APTUSDT, 1000PEPEUSDT]",0.802338,-0.496753,14.659084,7.713772,43.0,0.004240,-0.000413


In [20]:
merge_res.iloc[:10]

,signal,reverse,num_good_product,good_product,trainSharpe,testSharpe,trainAvgDailyTradeNum,testAvgDailyTradeNum,sum_good_strat,avgTrainRet,avgTestRet
5,kdj.k.32,-1,2,"[BTCUSDT, ETHUSDT]",4.229345,2.789503,50.783587,38.246154,NaN,0.003965,0.001580
24,kdj.j.128,-1,4,"[OPUSDT, BTCUSDT, ETHUSDT, 1000PEPEUSDT]",3.666122,1.707982,58.797775,39.661869,NaN,0.003684,0.000861
17,kdj.k.64,-1,3,"[BTCUSDT, ETHUSDT, 1000PEPEUSDT]",3.631148,1.868213,76.679746,49.007543,NaN,0.004547,-0.000224
20,range.pos.64,-1,5,"[OPUSDT, BTCUSDT, ETHUSDT, APTUSDT, 1000PEPEUSDT]",3.563911,2.062242,135.552122,95.312854,NaN,0.004581,0.000574
29,kdj.k.128,-1,5,"[OPUSDT, BTCUSDT, ETHUSDT, APTUSDT, 1000PEPEUSDT]",3.556452,1.641319,145.497059,103.410684,NaN,0.004715,0.000332
0,kdj.j.32,-1,3,"[BTCUSDT, ETHUSDT, 1000PEPEUSDT]",3.439240,2.502320,61.658335,46.711752,NaN,0.003915,0.000232
3,rmi.32,-1,5,"[OPUSDT, BTCUSDT, ETHUSDT, APTUSDT, 1000PEPEUSDT]",3.406843,2.353787,117.513401,70.882388,NaN,0.003414,0.000613
21,nr.64,-1,3,"[BTCUSDT, ETHUSDT, 1000PEPEUSDT]",3.347883,1.426225,40.768077,32.622255,NaN,0.007011,0.000928
12,kdj.j.64,-1,5,"[OPUSDT, BTCUSDT, ETHUSDT, APTUSDT, 1000PEPEUSDT]",3.324747,1.389937,172.042163,126.508276,NaN,0.006085,-0.000459
7,dbook.32,1,3,"[BTCUSDT, ETHUSDT, 1000PEPEUSDT]",3.302250,3.257246,47.878057,44.338761,48.0,0.003913,0.003272


# # (Direc With Var Signal)构造Portfolio后的Test Return Learderboard

In [12]:
results = factor_analysis.get_signal_performance_result(dire_signal_with_variance_period, "signal_result_with_atr", product_list, 1e-4, 1)

open file failed: /Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.32.reverse.pkl, e=[Errno 2] No such file or directory: '/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.32.reverse.pkl'
/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.32.reverse.pkl not found 
open file failed: /Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.64.trend.pkl, e=[Errno 2] No such file or directory: '/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.64.trend.pkl'
/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/1000PEPEUSDT.dbook.spread.adx.1024.2048.64.tre

KeyboardInterrupt: 

In [79]:
trend_res = results['trend.signal.stat']
reve_res = results["reverse.signal.stat"]
signal_trend_res = trend_res[['signal', 'trainSharpe']].set_index('signal')['trainSharpe']
signal_rever_res = reve_res[['signal', 'trainSharpe']].set_index('signal')['trainSharpe']

trend_signals = signal_trend_res[(signal_trend_res - signal_rever_res) > 0].index
reverse_signals = signal_trend_res[(signal_trend_res - signal_rever_res) < 0].index
nouse_signals = signal_trend_res[(signal_trend_res - signal_rever_res) == 0].index

print("#trend={}, #reverse={}, #nouse={}".format(len(trend_signals), len(reverse_signals), len(nouse_signals)))

merge_res = pd.concat([trend_res.loc[np.isin(trend_res['signal'], trend_signals)], 
                       reve_res.loc[np.isin(reve_res['signal'], reverse_signals)]], 
                      axis=0)

display(merge_res.sort_values(by='trainSharpe', ascending=False))

#trend=153, #reverse=155, #nouse=28


,signal,reverse,num_good_product,trainSharpe,testSharpe,trainAvgDailyTradeNum,testAvgDailyTradeNum,sum_good_strat,avgTrainRet,avgTestRet
20,kdj.k.std.2048.32,-1,1,5.522623,-1.860022,9.368421,9.548246,NaN,0.004960,-0.000775
91,dbook.spread.diff.adx.1024.2048.64,1,1,5.025148,0.121712,69.400000,28.467262,31.0,0.021904,0.000245
43,dbook.spread.diff.adx.1024.2048.32,1,1,4.954294,-0.377528,57.644898,24.960317,23.0,0.017055,-0.000973
155,adx.direc2.1024.adx.1024.2048.256,-1,1,4.750805,2.603924,5.745238,5.600000,NaN,0.009389,0.008191
99,kdj.j.adx.1024.2048.128,-1,5,3.815531,-0.254969,77.128873,57.193799,NaN,0.003774,-0.001422
...,...,...,...,...,...,...,...,...,...,...
325,nr.trend.index.2048.2048,-1,5,0.711335,1.572744,19.622001,19.064463,NaN,0.002161,0.002819
302,rmi.atr.2048.2048,1,4,0.700247,-0.809005,14.567847,8.108694,48.0,0.003025,-0.001288
202,adx.direc2.1024.atr.2048.512,1,5,0.679841,-0.779695,23.866640,10.322095,53.0,0.002899,-0.001319
277,nr.trend.index.2048.1024,-1,5,0.664394,1.100236,25.901751,24.541063,NaN,0.001839,0.001913


# # (Trades Direc)构造Portfolio后的Test Return Learderboard

In [13]:
results = factor_analysis.get_signal_performance_result(trades_signal_period, "signal_result_with_atr", product_list, 1e-4, 1)

trend_res = results['trend.signal.stat']
reve_res = results["reverse.signal.stat"]
signal_trend_res = trend_res[['signal', 'trainSharpe']].set_index('signal')['trainSharpe']
signal_rever_res = reve_res[['signal', 'trainSharpe']].set_index('signal')['trainSharpe']

trend_signals = signal_trend_res[(signal_trend_res - signal_rever_res) > 0].index
reverse_signals = signal_trend_res[(signal_trend_res - signal_rever_res) < 0].index
nouse_signals = signal_trend_res[(signal_trend_res - signal_rever_res) == 0].index

print("#trend={}, #reverse={}, #nouse={}".format(len(trend_signals), len(reverse_signals), len(nouse_signals)))

merge_res = pd.concat([trend_res.loc[np.isin(trend_res['signal'], trend_signals)], 
                       reve_res.loc[np.isin(reve_res['signal'], reverse_signals)]], 
                      axis=0)

display(merge_res.sort_values(by='trainSharpe', ascending=False))

open file failed: /Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/OPUSDT.trade.buy_power.more_position.12.1024.trend.pkl, e=[Errno 2] No such file or directory: '/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/OPUSDT.trade.buy_power.more_position.12.1024.trend.pkl'
/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/OPUSDT.trade.buy_power.more_position.12.1024.trend.pkl not found 
open file failed: /Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/BTCUSDT.trade.buy_power.more_position.12.1024.trend.pkl, e=[Errno 2] No such file or directory: '/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/BTCUSDT.trade.buy_power.more_position.12.1024.trend.pkl'
/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/BTCUSDT.trade.buy_power.more_position.12

,signal,reverse,num_good_product,trainSharpe,testSharpe,trainAvgDailyTradeNum,testAvgDailyTradeNum,sum_good_strat,avgTrainRet,avgTestRet
18,trade.buy_power_count.8192,1,1,5.813510,1.214648,6.145865,3.404386,19.0,0.026910,0.003936
23,trade.buy_power_count.16384,1,1,4.943631,1.135300,3.038571,1.703333,20.0,0.023128,0.002531
8,trade.buy_power_count.2048,1,3,3.174978,-0.380092,35.413672,20.998985,41.0,0.017576,-0.001223
3,trade.buy_power_count.1024,1,4,3.010763,-0.118609,84.312672,43.979480,55.0,0.013370,-0.000984
13,trade.buy_power_count.4096,1,3,2.988069,-0.105486,19.555106,12.543739,38.0,0.015988,-0.001230
21,trade.doublebuy.value.ratio.16384,1,4,2.764875,-1.756618,85.587831,24.951216,111.0,0.009557,-0.002563
16,trade.doublebuy.value.ratio.8192,1,4,2.408266,-1.896867,113.614481,35.594379,72.0,0.009876,-0.002637
6,trade.doublebuy.value.ratio.2048,1,4,1.734151,-1.724333,129.154607,33.817766,25.0,0.006041,-0.003301
1,trade.doublebuy.value.ratio.1024,-1,3,1.715036,1.105599,58.800214,22.180780,NaN,0.006928,0.002124
11,trade.doublebuy.value.ratio.4096,1,4,1.694909,-1.861340,131.970652,43.658393,55.0,0.007144,-0.002809
